In [51]:
using CSV
using DataFrames
using Plots
using Dates
using GLM
using Plots.PlotMeasures
ENV["COLUMNS"]=1000;
plotly()

Plots.PlotlyBackend()

In [2]:
df_fname = "scraper/autoscout.csv"
df = CSV.read(df_fname)
first(df, 3)

[176931:0503/085230.563682:ERROR:buffer_manager.cc(488)] [.DisplayCompositor]GL ERROR :GL_INVALID_OPERATION : glBufferData: <- error from previous GL command


,title,price,km,matriculation,fuel,euroclass,brand,model,doors,engine_displacement,hp,uuid,url
,String,Int64,Int64,String,String⍰,String⍰,String⍰,Missing,Int64⍰,Int64⍰,Int64,String,String
1,Fiat Panda 1.2 DIN 5P,2200,161200,2010-03-01 00:00:00,Benzina,Euro 5,Fiat,missing,5,1242,69,3b4ec4b6-04f5-49a2-8b7b-6f78fdec331e,https://www.autoscout24.it/annunci/fiat-panda-1-2-din-5p-benzina-blu-azzurro-3b4ec4b6-04f5-49a2-8b7b-6f78fdec331e
2,Fiat Panda 1.1 Active,2200,142803,2005-09-01 00:00:00,Benzina,Euro 4,Fiat,missing,5,1108,54,a8868f77-920e-4b33-89e9-6eac1d604019,https://www.autoscout24.it/annunci/fiat-panda-1-1-active-benzina-blu-azzurro-a8868f77-920e-4b33-89e9-6eac1d604019
3,Lancia Ypsilon,2200,150000,2004-04-01 00:00:00,Benzina,missing,missing,missing,3,missing,80,23e26b9c-749d-d369-e053-e250040ad537,https://www.autoscout24.it/annunci/lancia-ypsilon-benzina-blu-azzurro-23e26b9c-749d-d369-e053-e250040ad537


In [3]:
size(df)

(2150, 13)

In [4]:
df.fuel = Missings.coalesce.(df.fuel, "NA");

In [13]:
unique(df.fuel)

5-element Array{String,1}:
 "Gasoline"
 "LPG"
 "Metano"
 "NA"
 "Biogas"

In [10]:
function replace_with(it, to_be_replaced, replace_by)
    map(x -> (x ∈ to_be_replaced) ? replace_by : x, it)
end

replace_with (generic function with 1 method)

In [12]:
gasoline_like = ["Benzina", "Super 95", "Benzina (Filtro antiparticolato)", "Benzina E10 91", "Super 95 (Filtro antiparticolato)"]
df.fuel = replace_with(df.fuel, gasoline_like, "Gasoline");
lpg_like = ["GPL", "Gas di petrolio liquefatto", "Gas di petrolio liquefatto (Filtro antiparticolato)", "GPL (Filtro antiparticolato)"]
df.fuel = replace_with(df.fuel, lpg_like, "LPG");

In [15]:
by(df, [:fuel], nrow)

,fuel,x1
,String,Int64
1,Gasoline,1608
2,LPG,450
3,Metano,78
4,NA,11
5,Biogas,3


In [58]:
h1 = histogram(df.price, xlabel="Price €", bottom_margin = 15mm) 
h2 = histogram(df.hp, xlabel="Horsepower", bottom_margin = 15mm)
h3 = histogram(df.months_old, xlabel = "Months since matriculation", bottom_margin = 15mm)
h4 = histogram(df.km, xlabel = "mileage", bottom_margin = 15mm)
l = @layout [
     a{0.5h} b{0.5h}
     c{0.5h} d{0.5h}
]
plot(h1, h2, h3, h4, layout=l)

<!DOCTYPE html>
 
 
 Plots.jl

In [17]:
function filter_by_fuel(df, fueltype::String)
    filter(row -> row.fuel == fueltype, df)
end
function filter_by_fuel(df, fueltype::Array{String,1})
    filter(row -> row.fuel ∈ fueltype, df)
end

filter_by_fuel (generic function with 2 methods)

In [59]:
#df = dropmissing(df)
df = filter_by_fuel(df, ["LPG", "Gasoline"])
size(df)

(2058, 15)

In [60]:
dateformat = Dates.DateFormat("yyyy-mm-dd HH:MM:SS")
todatetime = x ->  Dates.DateTime(x, dateformat)
df.matriculation = todatetime.(df.matriculation);

MethodError: MethodError: no method matching Int64(::DateTime)
Closest candidates are:
  Int64(!Matched::Union{Bool, Int32, Int64, UInt32, UInt64, UInt8, Int128, Int16, Int8, UInt128, UInt16}) at boot.jl:707
  Int64(!Matched::Ptr) at boot.jl:717
  Int64(!Matched::Float32) at float.jl:707
  ...

In [22]:
function months_old(date)
    period = Dates.CompoundPeriod(convert(Dates.Millisecond, Dates.now() - date))
    can_period =Dates.canonicalize(period)
    weeks = can_period.periods[1].value 
    weeks / 4
end

months_old (generic function with 1 method)

In [72]:
df.months_old = months_old.(df.matriculation);
df.years_old = df.months_old / 12.0;

In [24]:
df[1:3, :]

,title,price,km,matriculation,fuel,euroclass,brand,model,doors,engine_displacement,hp,uuid,url,months_old,years_old
,String,Int64,Int64,DateTime,String,String⍰,String⍰,Missing,Int64⍰,Int64⍰,Int64,String,String,Float64,Float64
1,Fiat Panda 1.2 DIN 5P,2200,161200,2010-03-01T00:00:00,Gasoline,Euro 5,Fiat,missing,5,1242,69,3b4ec4b6-04f5-49a2-8b7b-6f78fdec331e,https://www.autoscout24.it/annunci/fiat-panda-1-2-din-5p-benzina-blu-azzurro-3b4ec4b6-04f5-49a2-8b7b-6f78fdec331e,132.5,11.0417
2,Fiat Panda 1.1 Active,2200,142803,2005-09-01T00:00:00,Gasoline,Euro 4,Fiat,missing,5,1108,54,a8868f77-920e-4b33-89e9-6eac1d604019,https://www.autoscout24.it/annunci/fiat-panda-1-1-active-benzina-blu-azzurro-a8868f77-920e-4b33-89e9-6eac1d604019,191.25,15.9375
3,Lancia Ypsilon,2200,150000,2004-04-01T00:00:00,Gasoline,missing,missing,missing,3,missing,80,23e26b9c-749d-d369-e053-e250040ad537,https://www.autoscout24.it/annunci/lancia-ypsilon-benzina-blu-azzurro-23e26b9c-749d-d369-e053-e250040ad537,209.75,17.4792


In [62]:
function filter_by_fuel(df, fueltype)
    filter(row -> row.fuel isa String && row.fuel == fueltype, df)
end

filter_by_fuel (generic function with 3 methods)

In [64]:
lpg_df = filter_by_fuel(df, "LPG")
gasoline_df = filter_by_fuel(df, "Gasoline");

In [65]:
scatter(lpg_df.km, lpg_df.years_old, lpg_df.price, xlabel="KM",
    ylabel="years old", zlabel="price", formatter = identity, color="yellow", label="LPG", marker=:square,
     size = (700, 700))
scatter!(gasoline_df.km, gasoline_df.years_old, gasoline_df.price, xlabel="KM", 
     ylabel="years old", zlabel="price", formatter = identity, color="green", label="gasoline", 
     )

<!DOCTYPE html>
 
 
 Plots.jl

In [67]:
mydf = filter_by_fuel(df, ["Gasoline", "LPG"]);

In [76]:
full_lm = lm(@formula(price ~ years_old + km + fuel + hp + brand ), mydf)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

price ~ 1 + years_old + km + fuel + hp + brand

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────────────────────
                           Estimate     Std. Error     t value  Pr(>|t|)      Lower 95%      Upper 95%
──────────────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)           10618.3         389.265        27.2779      <1e-99   9854.79       11381.9
years_old               -61.4696        6.82911      -9.00112     <1e-18    -74.8652       -48.074
km                       -0.0430407     0.00128278  -33.5528      <1e-99     -0.0455569     -0.0405244
fuel: LPG               450.17        145.683         3.09007     0.0020    164.406        735.933
hp                       30.8835        1.62045      19.0587 

In [78]:
r2(full_lm)

0.5945868229020579

In [ ]:
lmres = lm(@formula(price ~ years_old), df)
years_old = range(2, 11, length=100)

scatter(df.years_old, df.price)
plot!(years_old, predict(lmres, DataFrame(years_old=years_old)))


In [ ]:
r2(lmres)

In [ ]:
filter(x -> x.price > 2000 && x.price < 4000 && x.years_old < 10 , df)